In [1]:
import random
import numpy as np
from tqdm import tqdm


In [2]:
class Graphe:
    def __init__(self):
        return
    
    def creasommet(self, n, distance_max):
        # Créer une matrice de distances
        distances = [[(0, 1.0) if i == j else None for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(i + 1, n):
                distance = random.randint(1, distance_max)
                pheromone = 1.0  # Initial pheromone level
                distances[i][j] = (distance, pheromone)
                distances[j][i] = (distance, pheromone)
        return distances
    
    def creawindow(self, distance, duree_min, duree_max):
        # Créer une fenêtre de durée
        debut = int(distance)
        duree = random.randint(duree_min, duree_max)
        fin = debut + duree
        return (debut, fin)


# Initialisation du graphe

In [3]:
n = 10000
distance_max = n * 3
duree_min = 5
duree_max = 20

In [4]:
# Paramètres pour l'algorithme des colonies de fourmis (ACO)

# Nombre de fourmis dans la colonie
nombres_fourmis = 10  # Plus de fourmis augmentent l'exploration mais aussi le coût de calcul

# Paramètre d'importance de la phéromone
alpha = 1  # Plus alpha est élevé, plus les fourmis suivent les traces de phéromones

# Paramètre d'importance de l'heuristique (ex. : distance, attractivité)
beta = 2  # Plus beta est élevé, plus les fourmis privilégient les chemins courts

# Seuil pour probabilité de choix aléatoire (ou exploration/ intensification)
y = 0.1  # Contrôle la probabilité de diversification de la recherche

# Taux d'évaporation des phéromones
evaporation = 0.1  # Plus l'évaporation est rapide, moins les anciennes solutions influencent la recherche

# Quantité de phéromones déposée par chaque fourmi à chaque cycle
Q = 2  # Contrôle l'intensité des traces de phéromones laissées par chaque fourmi


## Creation bibliothéque

In [5]:
graphe = Graphe()
distances = graphe.creasommet(n, distance_max)

fenetres = [[(0, 0) for _ in range(n)] for _ in range(n)]

for i in range(n):
    for j in range(i + 1, n):
        distance_ij = distances[i][j]
        fenetre_ij = graphe.creawindow(distance_ij[0], duree_min, duree_max)
        fenetres[i][j] = fenetre_ij
        fenetres[j][i] = fenetre_ij




## Solution initiale

In [6]:
liste_adjacence = {i: {j: distances[i][j] for j in range(n) if distances[i][j] is not None} for i in range(n)}

fenetres_dict = {
    i: {j: fenetres[i][j] for j in range(n)} 
    for i in range(n)
}

## Meta heuristique Fourmi

In [7]:
import numpy as np
from tqdm import tqdm

class ACO:
    def __init__(self):
        return

    def parcours(self, villes):
        trajet = [0]
        heure_actuelle = 0
        compteur = 0
        visited = set(trajet)

        with tqdm(total=n, desc="Itérations de parcours") as pbar:
            while len(trajet) < n:
                if compteur == 200:
                    distance_to_start = villes[trajet[-1]][0][0]
                    heure_actuelle += distance_to_start
                    trajet.append(0)
                    visited.clear()
                    visited.add(0)
                    compteur = 0
                    continue

                voisins = [ville for ville in villes[trajet[-1]] if ville not in visited]

                if not voisins:
                    break

                distances = np.array([villes[trajet[-1]][ville][0] for ville in voisins])
                pheromones = np.array([villes[trajet[-1]][ville][1] for ville in voisins])
                debuts = np.array([fenetres_dict[trajet[-1]][ville][0] for ville in voisins])
                fins = np.array([fenetres_dict[trajet[-1]][ville][1] for ville in voisins])

                valid = (distances >= debuts) & (distances <= fins)
                probabilites = (valid * ((pheromones ** alpha) * (1 / distances ** beta)))
                somme_probabilites = probabilites.sum()

                if somme_probabilites == 0:
                    break

                probabilites /= somme_probabilites
                prochain = np.random.choice(voisins, p=probabilites)
                trajet.append(prochain)
                visited.add(prochain)

                heure_actuelle += villes[trajet[-2]][prochain][0]
                compteur += 1
                pbar.update(1)

        trajet.append(trajet[0])
        self.add_pheromone(trajet)

    def add_pheromone(self, trajet):
        for i in range(len(trajet) - 1):
            ville_actuelle = trajet[i]
            ville_suivante = trajet[i + 1]
            distance, pheromone_actuel = liste_adjacence[ville_actuelle][ville_suivante]
            nouvelle_pheromone = pheromone_actuel + Q / distance
            liste_adjacence[ville_actuelle][ville_suivante] = (distance, nouvelle_pheromone)
            liste_adjacence[ville_suivante][ville_actuelle] = (distance, nouvelle_pheromone)

        self.add_evaporation()

    def add_evaporation(self):
        for ville in liste_adjacence:
            for voisin in liste_adjacence[ville]:
                dist, pher = liste_adjacence[ville][voisin]
                liste_adjacence[ville][voisin] = (dist, pher * (1 - evaporation))

# Configuration de l'ACO et exécution
aco = ACO()
with tqdm(total=nombres_fourmis, desc="Fourmis parcours") as fourmis_bar:
    for _ in range(nombres_fourmis):
        aco.parcours(liste_adjacence)
        fourmis_bar.update(1)

# Trouver le meilleur trajet parmi les fourmis
trajet = [0]
distance_all = 0
compteur = 0
visited = set(trajet)

while len(trajet) < (len(liste_adjacence) + n / 200):
    if compteur == 200:
        trajet.append(0)
        distance_all += liste_adjacence[trajet[-1]][0][0]
        visited.clear()
        visited.add(0)
        compteur = 0

    voisins = {ville: valeur for ville, valeur in liste_adjacence[trajet[-1]].items() if ville not in visited}
    if not voisins:
        break

    prochain_voisin = min(voisins, key=lambda x: voisins[x][0])
    distance, _ = liste_adjacence[trajet[-1]][prochain_voisin]
    debut, fin = fenetres_dict[trajet[-1]][prochain_voisin]

    if debut <= distance <= fin:
        trajet.append(prochain_voisin)
        visited.add(prochain_voisin)
        distance_all += distance
        compteur += 1

trajet.append(0)
distance_all += liste_adjacence[trajet[-1]][0][0]


Fourmis parcours: 100%|██████████| 10/10 [2:20:07<00:00, 840.79s/it]


# Affichage des résultats


In [8]:
print("Trajet: ", trajet)
print("Distance totale: ", distance_all)
print(len(trajet))

Trajet:  [0, 8880, 6152, 3151, 7428, 5576, 5328, 6928, 3852, 522, 3496, 612, 4660, 9983, 7325, 7566, 3907, 1523, 423, 7846, 2760, 4967, 8443, 6794, 9877, 7766, 7006, 2758, 3695, 4187, 1195, 4441, 6044, 8431, 9523, 5020, 6434, 5280, 4795, 3022, 5646, 877, 5096, 5297, 9229, 5256, 8023, 2642, 4417, 2742, 5607, 1322, 5558, 7326, 4206, 2585, 2524, 3373, 4872, 237, 302, 9828, 1179, 1879, 5954, 9636, 5282, 5277, 3488, 4979, 2346, 1936, 2587, 5739, 2367, 9660, 9520, 7459, 7088, 3067, 7065, 77, 4492, 7265, 222, 6877, 8412, 8396, 8205, 7935, 3137, 3813, 525, 5379, 1219, 3063, 1281, 8038, 7658, 7727, 6775, 8551, 6500, 6705, 7146, 1542, 3717, 1183, 3989, 204, 6873, 8009, 2387, 860, 7107, 4706, 4371, 19, 3875, 2298, 1330, 2461, 2503, 8944, 1245, 5237, 9975, 1092, 9210, 2232, 3436, 6984, 1858, 5923, 6939, 8267, 3267, 2189, 7304, 5064, 2342, 2738, 2841, 8405, 5197, 7185, 2626, 771, 2691, 1877, 2097, 5426, 8082, 6813, 1604, 3741, 541, 8781, 2469, 5369, 7120, 1210, 1995, 1307, 4020, 3253, 2356, 516, 46